In [ ]:
%load_ext autoreload
%autoreload 2

## Basic setup

In [ ]:
import Concord as ccd
import scanpy as sc
import torch
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

data_dir = Path('../data/celegans_data/')
data_path = data_dir / "celegans_global_adata.h5ad"
# adata = sc.read(
#     data_path
# )
adata = sc.read(data_dir/'celegans_Dec17-2043.h5ad')

In [ ]:
import time
from pathlib import Path
proj_name = "concord_celegans"
save_dir = f"../save/dev_{proj_name}-{time.strftime('%b%d')}/"
save_dir = Path(save_dir)
save_dir.mkdir(parents=True, exist_ok=True)
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
file_suffix = f"{time.strftime('%b%d-%H%M')}"
seed = 0

In [ ]:
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata = adata[adata.obs['to.filter'] == 'FALSE']

## Run Concord

In [ ]:
feature_list = ccd.ul.select_features(adata, n_top_features=10000, flavor='seurat_v3')
file_suffix = f"{time.strftime('%b%d-%H%M')}"
concord_args = {
        'adata': adata,
        'input_feature': feature_list,
        'batch_size': 128,
        'latent_dim': 300,
        'encoder_dims':[1000],
        'decoder_dims':[1000],
        'augmentation_mask_prob': 0.3, 
        'clr_temperature': 0.5,
        'p_intra_knn': 0.3,
        'sampler_knn': 300,
        'min_p_intra_domain': .95,
        'n_epochs': 15,
        'domain_key': 'batch',
        'verbose': False,
        'inplace': False,
        'seed': seed,
        'device': device,
        'save_dir': save_dir
    }

### Concord, no decoder

In [ ]:
output_key = f'Concord_{file_suffix}'
cur_ccd = ccd.Concord(use_decoder=False, **concord_args)
cur_ccd.encode_adata(input_layer_key='X_log1p', output_key=output_key)
# Save the latent embedding to a filem, so that it can be loaded later
ccd.ul.save_obsm_to_hdf5(cur_ccd.adata, save_dir / f"obsm_{file_suffix}.h5")
adata.obsm = cur_ccd.adata.obsm # If not inplace

In [ ]:
basis = output_key
ccd.ul.run_umap(adata, source_key=basis, result_key=f'{basis}_UMAP', n_components=2, n_neighbors=30, min_dist=0.1, metric='euclidean', random_state=seed)
show_basis = basis + '_UMAP'
show_cols = ['cell.type', 'raw.embryo.time', "lineage"]
pal = {'cell.type': 'tab20', 'raw.embryo.time': 'BlueGreenRed', "lineage": 'tab20'}
ccd.pl.plot_embedding(
    adata, show_basis, show_cols, figsize=(13,5), dpi=600, ncols=3, font_size=5, point_size=1, legend_loc=None,
    pal = pal,
    save_path=save_dir / f"{show_basis}_{file_suffix}.png"
)

In [ ]:
ccd.ul.run_umap(adata, source_key=basis, result_key=f'{basis}_UMAP_3D', n_components=3, n_neighbors=30, min_dist=0.1, metric='euclidean', random_state=seed)
for col in show_cols:
    show_basis = f'{basis}_UMAP_3D'
    ccd.pl.plot_embedding_3d(
            adata, basis=show_basis, color_by=col,
            pal = pal[col],
            save_path=save_dir / f'{show_basis}_{col}_{file_suffix}.html',
            point_size=1, opacity=0.8, width=1500, height=1000
        )

### Concord, with decoder

In [ ]:
output_key = f'Concord-decoder_{file_suffix}'
cur_ccd = ccd.Concord(use_decoder=True, **concord_args)
cur_ccd.encode_adata(input_layer_key='X_log1p', output_key=output_key)
# Save the latent embedding to a filem, so that it can be loaded later
ccd.ul.save_obsm_to_hdf5(cur_ccd.adata, save_dir / f"obsm_{file_suffix}.h5")
adata.obsm = cur_ccd.adata.obsm # If not inplace

In [ ]:
output_key = 'Concord-decoder_Dec15-1329'
basis = output_key
#ccd.ul.run_umap(adata, source_key=basis, result_key=f'{basis}_UMAP', n_components=2, n_neighbors=30, min_dist=0.1, metric='euclidean', random_state=seed)
show_basis = basis + '_UMAP'
show_cols = ['cell.type', 'raw.embryo.time', "lineage"]
pal = {'cell.type': 'tab20', 'raw.embryo.time': 'BlueGreenRed', "lineage": 'tab20'}
ccd.pl.plot_embedding(
    adata, show_basis, show_cols, figsize=(13,5), dpi=600, ncols=3, font_size=5, point_size=1, legend_loc=None,
    pal = pal,
    save_path=save_dir / f"{show_basis}_{file_suffix}.png"
)

In [ ]:
ccd.ul.run_umap(adata, source_key=basis, result_key=f'{basis}_UMAP_3D', n_components=3, n_neighbors=30, min_dist=0.1, metric='euclidean', random_state=seed)
for col in show_cols:
    show_basis = f'{basis}_UMAP_3D'
    ccd.pl.plot_embedding_3d(
            adata, basis=show_basis, color_by=col,
            pal = pal[col],
            save_path=save_dir / f'{show_basis}_{col}_{file_suffix}.html',
            point_size=1, opacity=0.8, width=1500, height=1000
        )

In [ ]:
adata = sc.read(data_dir / f"celegans_{file_suffix}.h5ad")

In [ ]:
adata.obsm

### PAGA analysis

In [ ]:
## Run leiden clustering on the latent
sc.pp.neighbors(adata, n_neighbors=10, use_rep='Concord_Dec16-1646')
sc.tl.leiden(adata, resolution=5.0, key_added='leiden_Concord', random_state=seed)

In [ ]:
## Run leiden clustering on the latent
sc.pp.neighbors(adata, n_neighbors=30, use_rep='Concord_Dec16-1646')
sc.tl.leiden(adata, resolution=0.2, key_added='leiden_Concord_broad', random_state=seed)

In [ ]:
adata.obs['lineage_ctype'] = adata.obs['lineage'].copy().astype(str)
adata.obs['lineage_ctype'][adata.obs['lineage'] == 'unannotated'] = adata.obs['plot.cell.type'][adata.obs['lineage'] == 'unannotated'].astype(str)

In [ ]:
output_key = 'Concord_Dec16-1646'
basis = output_key
#ccd.ul.run_umap(adata, source_key=basis, result_key=f'{basis}_UMAP', n_components=2, n_neighbors=30, min_dist=0.1, metric='euclidean', random_state=seed)
show_basis = basis + '_UMAP'
show_cols = ['raw.embryo.time', 'leiden_Concord', 'lineage_ctype', 'cell.type', 'leiden_Concord_broad']
pal = {'leiden_Concord': 'tab20', 'raw.embryo.time': 'BlueGreenRed', "lineage_ctype": 'tab20', 'cell.type': 'tab20', 'leiden_Concord_broad': 'tab20'}
ccd.pl.plot_embedding(
    adata, show_basis, show_cols, figsize=(13,8), dpi=600, ncols=3, font_size=5, point_size=1, legend_loc='on data',
    pal = pal,
    save_path=save_dir / f"{show_basis}_{file_suffix}.png"
)

In [ ]:
ccd.ul.ensure_categorical(adata, 'leiden_Concord')
sc.tl.paga(adata, groups='leiden_Concord')

In [ ]:
#data_col, cmap, palette = ccd.pl.get_color_mapping(adata, color_by, pal)
color_by = 'leiden_Concord'
threshold=0.5
node_size_scale=0.3
edge_width_scale = 0.05
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
sc.pl.paga(
            adata, threshold=threshold, color=color_by, ax=ax, show=False,
            layout='fa', fontsize=2, cmap=None, node_size_scale=node_size_scale,
            edge_width_scale=edge_width_scale, colorbar=False
        )
plt.savefig(save_dir / f"paga_{color_by}_{file_suffix}.png")

In [ ]:
ccd.ul.compute_meta_attributes(
    adata,
    groupby_key='leiden_Concord',
    attribute_key='lineage_ctype',
    method='majority_vote',
    meta_label_name='meta_lineage'
)
ccd.ul.compute_meta_attributes(
    adata,
    groupby_key='leiden_Concord',
    attribute_key='cell.type',
    method='majority_vote',
    meta_label_name='meta_celltype'
)

ccd.ul.compute_meta_attributes(
    adata,
    groupby_key='leiden_Concord',
    attribute_key='raw.embryo.time',
    method='average',
    meta_label_name='meta_time'
)
# Convert meta_time to numeric
adata.obs['meta_time'] = adata.obs['meta_time'].astype(float)

adata.obs['broad_lineage'] = adata.obs['lineage'].copy().astype(str)
# Take the first 5 characters of the lineage
adata.obs['broad_lineage'] = adata.obs['broad_lineage'].str[:5]
ccd.ul.compute_meta_attributes(
    adata,
    groupby_key='leiden_Concord',
    attribute_key='broad_lineage',
    method='majority_vote',
    meta_label_name='meta_broad_lineage'
)

adata.obs['broad_lineage_ctype'] = adata.obs['broad_lineage'].copy().astype(str)
adata.obs['broad_lineage_ctype'][adata.obs['broad_lineage'] == 'unann'] = adata.obs['cell.type'][adata.obs['broad_lineage'] == 'unann'].astype(str)
ccd.ul.compute_meta_attributes(
    adata,
    groupby_key='leiden_Concord',
    attribute_key='broad_lineage_ctype',
    method='majority_vote',
    meta_label_name='meta_broad_lineage_ctype'
)

In [ ]:
filtered_graph, pos = ccd.pl.compute_paga_layout(
    adata,
    groupby_key='leiden_Concord',
    weight_threshold=0.8,
    spring_k=0.05,
    seed=1
)

In [ ]:
meta_key = 'meta_time'
ccd.pl.plot_paga_custom(
    adata,
    meta_attribute_key=meta_key,
    groupby_key='leiden_Concord',
    filtered_graph=filtered_graph,
    pos=pos,
    with_labels=False,
    pal={'meta_time': 'BlueGreenRed'},
    figsize=(20,20),
    save_path=save_dir / f"paga_graph_{meta_key}_{file_suffix}.svg"
)

In [ ]:
meta_key = 'meta_broad_lineage_ctype'
ccd.pl.plot_paga_custom(
    adata,
    meta_attribute_key='meta_broad_lineage_ctype',
    groupby_key='leiden_Concord',
    filtered_graph=filtered_graph,
    pos=pos,
    with_labels=True,
    pal={meta_key: 'tab20'},
    figsize=(20,20),
    save_path=save_dir / f"paga_graph_{meta_key}_{file_suffix}.svg"
)

In [ ]:

adata.write_h5ad(data_dir / f"celegans_{file_suffix}.h5ad")

file_suffix

### Subgroup analysis

### Early 250 min

In [ ]:
sub_name = 'early250_noE'
adata_sub = adata[(adata.obs['raw.embryo.time'] <= 250) & (adata.obs['cell.type'].astype(str) != 'Intestine')]
print(adata_sub.shape)
## Run leiden clustering on the latent
cluster_key = f'leiden_{sub_name}'
sc.pp.neighbors(adata_sub, n_neighbors=10, use_rep='Concord_Dec16-1646')
sc.tl.leiden(adata_sub, resolution=8.0, key_added=cluster_key, random_state=seed)

In [ ]:
output_key = 'Concord_Dec16-1646'
basis = output_key
ccd.ul.run_umap(adata_sub, source_key=basis, result_key=f'{basis}_UMAP', n_components=2, n_neighbors=30, min_dist=0.1, metric='euclidean', random_state=seed)
show_basis = basis + '_UMAP'
show_cols = ['raw.embryo.time', cluster_key, 'lineage']
pal = {cluster_key: 'tab20', 'raw.embryo.time': 'BlueGreenRed', "lineage": 'tab20'}
ccd.pl.plot_embedding(
    adata_sub, show_basis, show_cols, figsize=(13,5), dpi=600, ncols=3, font_size=5, point_size=10, legend_loc=None,
    pal = pal,
    save_path=save_dir / f"{show_basis}_{file_suffix}.png"
)

In [ ]:
ccd.ul.ensure_categorical(adata_sub, cluster_key)
sc.tl.paga(adata_sub, groups=cluster_key)
filtered_graph, pos = ccd.pl.compute_paga_layout(
    adata_sub,
    groupby_key=cluster_key,
    weight_threshold=0.6,
    spring_k=0.02,
    seed=1
)

In [ ]:
meta_key = f'meta_lineage_{sub_name}'
ccd.ul.compute_meta_attributes(
    adata_sub,
    groupby_key=cluster_key,
    attribute_key='lineage',
    method='majority_vote',
    meta_label_name=meta_key
)
ccd.pl.plot_paga_custom(
    adata_sub,
    meta_attribute_key=meta_key,
    groupby_key=cluster_key,
    filtered_graph=filtered_graph,
    pos=pos,
    with_labels=True,
    pal={meta_key: 'tab20'},
    figsize=(20,20),
    save_path=save_dir / f"paga_graph_{meta_key}_{file_suffix}.svg"
)

In [ ]:
meta_key = f'meta_broad_lineage_{sub_name}'
adata_sub.obs['broad_lineage'] = adata_sub.obs['lineage'].copy().astype(str)
# Take the first 5 characters of the lineage
adata_sub.obs['broad_lineage'] = adata_sub.obs['broad_lineage'].str[:5]
ccd.ul.compute_meta_attributes(
    adata_sub,
    groupby_key=cluster_key,
    attribute_key='broad_lineage',
    method='majority_vote',
    meta_label_name=meta_key
)
ccd.pl.plot_paga_custom(
    adata_sub,
    meta_attribute_key=meta_key,
    groupby_key=cluster_key,
    filtered_graph=filtered_graph,
    pos=pos,
    with_labels=True,
    pal={meta_key: 'tab20'},
    figsize=(20,20),
    save_path=save_dir / f"paga_graph_{meta_key}_{file_suffix}.svg"
)

### Muscle

In [ ]:
sub_name = 'muscle'
adata_sub = adata[(adata.obs['cell.type'].astype(str) == 'Body_wall_muscle')]
print(adata_sub.shape)
## Run leiden clustering on the latent
cluster_key = f'leiden_{sub_name}'
sc.pp.neighbors(adata_sub, n_neighbors=10, use_rep='Concord_Dec16-1646')
sc.tl.leiden(adata_sub, resolution=8.0, key_added=cluster_key, random_state=seed)

In [ ]:
rare_lineage = (adata_sub.obs['lineage'].value_counts()[adata_sub.obs['lineage'].value_counts() < 5]).index.astype(str)
rare_lineage

In [ ]:
output_key = 'Concord_Dec16-1646'
basis = output_key
ccd.ul.run_umap(adata_sub, source_key=basis, result_key=f'{basis}_UMAP', n_components=2, n_neighbors=30, min_dist=0.1, metric='euclidean', random_state=1)
show_basis = basis + '_UMAP'
show_cols = ['raw.embryo.time', cluster_key, 'cell.type', 'cell.subtype', 'lineage', 'lineage_ctype']
pal = {cluster_key: 'tab20', 'raw.embryo.time': 'BlueGreenRed', "cell.type": 'tab20', 'cell.subtype':'tab20', 'lineage': 'tab20', 'lineage_ctype': 'tab20'}
ccd.pl.plot_embedding(
    adata_sub[~adata_sub.obs['lineage'].isin(rare_lineage)], show_basis, show_cols, figsize=(13,8), dpi=600, ncols=3, font_size=5, point_size=10, legend_loc='on data',
    pal = pal,
    save_path=save_dir / f"{show_basis}_{file_suffix}.svg"
)

In [ ]:
ccd.ul.ensure_categorical(adata_sub, cluster_key)
sc.tl.paga(adata_sub, groups=cluster_key)
filtered_graph, pos = ccd.pl.compute_paga_layout(
    adata_sub,
    groupby_key=cluster_key,
    weight_threshold=0.6,
    spring_k=0.02,
    seed=1
)

In [ ]:
meta_key = f'meta_lineage_ctype_{sub_name}'
ccd.ul.compute_meta_attributes(
    adata_sub,
    groupby_key=cluster_key,
    attribute_key='lineage_ctype',
    method='majority_vote',
    meta_label_name=meta_key
)
ccd.pl.plot_paga_custom(
    adata_sub,
    meta_attribute_key=meta_key,
    groupby_key=cluster_key,
    filtered_graph=filtered_graph,
    pos=pos,
    with_labels=True,
    pal={meta_key: 'tab20'},
    figsize=(20,20),
    save_path=save_dir / f"paga_graph_{meta_key}_{file_suffix}.svg"
)

In [ ]:
# Save adata_sub


In [ ]:

import numpy as np
import scanpy as sc
import time
from pathlib import Path
import torch
import Concord as ccd
import warnings
warnings.filterwarnings('ignore')

print("Import successful")

proj_name = "cbce_1217"
data_dir = Path('../../data/CE_CB/')
save_dir = f"../../save/dev_{proj_name}-{time.strftime('%b%d')}/"
save_dir = Path(save_dir)
save_dir.mkdir(parents=True, exist_ok=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
seed = 0
ccd.ul.set_seed(seed)

file_suffix = f"{time.strftime('%b%d-%H%M')}"


adata = sc.read(data_dir / "adata_cbce_Dec17-1824.h5ad")
adata.obsm = ccd.ul.load_obsm_from_hdf5(Path('../../save/dev_cbce_1217-Dec18/') / f"obsm_Dec18-1227.h5")
print("Data loading successful")

file_suffix = f"{time.strftime('%b%d-%H%M')}"

ccd.set_verbose_mode(True)
timer = ccd.ul.Timer()
time_log = {}


# Run methods
feature_list = ccd.ul.select_features(adata, n_top_features=10000, flavor='seurat_v3')
adata = adata[:,feature_list].copy()
output_key = 'Liger'
with timer:
    ccd.ul.run_liger(adata, batch_key="dataset3", count_layer="counts", output_key=output_key, k=30, return_corrected=False)

time_log[output_key] = timer.interval
ccd.ul.save_obsm_to_hdf5(adata, save_dir / f"obsm_{output_key}_{file_suffix}.h5")
    
# Save time_log as well using pickle
import pickle
import pickle
with open(save_dir / f"time_log_{output_key}_{file_suffix}.pkl", 'wb') as f:
    pickle.dump(time_log, f)



